<a href="https://colab.research.google.com/github/avshmelev/airflow-docker/blob/main/Airflow.%20Message%20to%20Slack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Настройка окружения

Для начала вам необходимо выполнить ряд команд чтобы настроить окружение для дальнейшей работы, это позволит первое время не заниматься долгим деплоем, а сразу начать писать код и работать с airflow.

In [ ]:
# Установка Airflow
!pip install apache-airflow==2.1.4
!pip install apache-airflow-providers-http
#####
##### ВАШ КОД Установите Slack Operator
#####
!pip install apache-airflow[slack]
!airflow db init

In [2]:
# Создадим папку dags
# В этой папке лежат скрипты для создания дагов
# Это стандартное имя для  данной папки
!mkdir /root/airflow/dags
!touch /root/airflow/dags/dag.py

In [ ]:
# Последующие команды не имеют отношения к Airflow
# Они нужни только для корректоной работы веб приложения
# в среде Google Colab

!pip install pyngrok

!ngrok authtoken 1yMOS9zzS85dgM5tk81bQkkL7Lq_6YMxq7UbkBK6LMWPmuPNd # найти его можно https://dashboard.ngrok.com/get-started/setup 

In [4]:
!airflow users create \
          --username admin \
          --firstname admin \
          --lastname admin \
          --role Admin \
          --email admin@example.org \
          -p 12345

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144 UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
[2021-10-31 20:50:01,506] {manager.py:798} WARNING - No user yet created, use flask fab command to do it.
Admin user admin created


In [5]:
# Включим веб-сервер
!airflow webserver -p 18273 -D

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144 UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  ____________       _____________
 ____    |__( )_________  __/__  /________      __
____  /| |_  /__  ___/_  /_ __  /_  __ \_ | /| / /
___  ___ |  / _  /   _  __/ _  / / /_/ /_ |/ |/ /
 _/_/  |_/_/  /_/    /_/    /_/  \____/____/|__/
[2021-10-31 20:50:48,496] {dagbag.py:496} INFO - Filling up the DagBag from /dev/null
Running the Gunicorn Server with:
Workers: 4 sync
Host: 0.0.0.0:18273
Timeout: 120
Logfiles: - -
Access Logformat: 


In [6]:
# Запуск шедулера
!airflow scheduler -D

  ____________       _____________
 ____    |__( )_________  __/__  /________      __
____  /| |_  /__  ___/_  /_ __  /_  __ \_ | /| / /
___  ___ |  / _  /   _  __/ _  / / /_/ /_ |/ |/ /
 _/_/  |_/_/  /_/    /_/    /_/  \____/____/|__/


In [7]:
# Эта команда просто отображет веб морду на другой адрес
# Его вы можете найти https://dashboard.ngrok.com/endpoints/status
# При каждом отключении ссылка будет меняться
!nohup ngrok http -log=stdout 18273 > /dev/null &

nohup: redirecting stderr to stdout


### Задача на разработку

Необходимо реализовать PythonOperator который будет бросать исключение raise AirflowException самостоятельно изучите [данный](https://airflow.apache.org/docs/apache-airflow/stable/_modules/airflow/exceptions.html) материал. После этого реализуйте функцию которая будет отправлять сообщение в Slack при failed, решение задачи можно взять вот [тут](https://medium.com/datareply/integrating-slack-alerts-in-airflow-c9dcd155105). Нужно создать Slack пространство.

In [ ]:
## ВАШЕ РЕШЕНИЕ
from airflow import DAG
from datetime import datetime
from airflow.utils.dates import days_ago
from airflow.hooks.base_hook import BaseHook
from airflow.operators.python import PythonOperator
from airflow.contrib.operators.slack_webhook_operator import SlackWebhookOperator
from airflow.exceptions import AirflowException


SLACK_CONN_ID = 'slack'


def raise_airflow_exception():
  raise AirflowException


def callback_if_failed(context):
  slack_token = BaseHook.get_connection(SLACK_CONN_ID).password
  slack_msg = f'Task Failed\n'  \
              f'Task: {context.get("task_instance").task_id}\n'  \
              f'DAG: {context.get("task_instance").dag_id}\n'  \
              f'Excec_date: {context.get("execution_date")}\n'  \
              f'Log_url: {context.get("task_instance").log_url}'

  failed_alert = SlackWebhookOperator(
      task_id = 'send_slack_msg',
      http_conn_id = 'slack',
      webhook_token = slack_token,
      message = slack_msg,      
      username = 'airflow'
  )

  return failed_alert.execute(context=context)


dag = DAG(
          dag_id='slack_dag',
          on_failure_callback=callback_if_failed,
          schedule_interval='@daily',
          start_date=days_ago(5))

failed_task = PythonOperator(
    task_id='raise_exception',
    python_callable=raise_airflow_exception,
    dag=dag
)